In [5]:
pip install numpy pandas requests xlsxwriter python-math yfinance

Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
import yfinance as yf

In [7]:
# Get the portfolio size from the user
portfolio_size = float(input("Enter the total portfolio size in dollars: "))

Enter the total portfolio size in dollars:  1000000


In [27]:
# Calculate the number of shares for an equal weighted portfolio
def equal_weighted_portfolio(result_df):
    num_tickers = len(result_df)
    equal_weight = float(portfolio_size) / float(num_tickers)
    
    result_df['Number of Shares'] = round(result_df['Last Price'].apply(
        lambda x: equal_weight / x if x else None)
    )
    return result_df

In [28]:
# Calculate the number of shares in market capital weighted portfolio
def mcap_weighted_portfolio (result_df):
    total_market_cap = result_df['Market Cap'].sum()
    result_df['Weight'] = result_df['Market Cap'] / total_market_cap
    result_df['Dollar Amount Allocated'] = result_df['Weight'] * portfolio_size
    result_df['Number of Shares'] = round( result_df.apply(
        lambda row: row['Dollar Amount Allocated'] / row['Last Price'] if row['Last Price'] else None, axis=1
    ))
    return result_df

In [10]:
# Get the file path to file with list of tickers in our portfolio
file_path = str(input("Enter the File path for CSV file to stock with stock tickers.: "))

Enter the File path for CSV file to stock with stock tickers.:  https://raw.githubusercontent.com/veejay-kathuria/portfolio_weights/main/example%20file.csv


In [11]:
# Getting type of portfolio weighting
portfolio_type = input("Enter '1' for an Market capital weighted portfolio else press enter. ")

Enter '1' for an Market capital weighted portfolio else press enter.  1


In [29]:
# Load the tickers from the Excel sheet
tickers_df = pd.read_csv(file_path)
tickers = tickers_df['Ticker'].tolist()

In [30]:
# Prepare a list to hold the data
data = []
# Fetching market cap and last price for each ticker
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        market_cap = info.get('marketCap')
        last_price = info.get('previousClose')
        d = [ticker, market_cap, last_price]
        data.append([ticker, market_cap, last_price])
    except Exception as e:
        print(f"Could not retrieve data for {ticker}: {e}")
        data.append([ticker, None, None])
        continue

# Create a DataFrame from the data
result_df = pd.DataFrame(data, columns=['Ticker', 'Market Cap', 'Last Price'])

In [43]:
#Calculate number of shares of each ticker as per type of weighting
result_df = equal_weighted_portfolio (result_df) if (portfolio_type != 1) else mcap_weighted_portfolio(result_df)

In [42]:
# Save the result to a new Excel sheet
result_file_path = 'sp_500_stocks_with_market_cap_and_shares.xlsx'
result_df.to_csv(result_file_path, index=False)

print(f"Data saved to {result_file_path}")

Data saved to sp_500_stocks_with_market_cap_and_shares.xlsx
